<a href="https://colab.research.google.com/github/amina200003/projet_academique/blob/main/pain_ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 7.1 MB/s eta 0:00:00


In [ ]:

from rdflib import Graph, Literal, Namespace, RDF, RDFS, OWL, URIRef, XSD

graph = Graph()
pain= Namespace("http://example.org/ontopain#")

graph.bind("pain",pain)


#Fonction pour créer une propriété objet
def create_object_property(name, domain=None, range_=None):
    prop = pain[name]
    graph.add((prop, RDF.type, OWL.ObjectProperty))
    if domain:
        graph.add((prop, RDFS.domain, pain[domain]))
    if range_:
        graph.add((prop, RDFS.range, pain[range_]))
    return prop

# Fonction pour créer une classe
def create_class(name, parent=None, label_fr=None, comment_fr=None):
    cls = pain[name]
    graph.add((cls, RDF.type, OWL.Class))
    if name != "Pain":  # Ne pas ajouter une parenté à soi-même
        parent_class = pain[parent] if parent else pain.Pain
        graph.add((cls, RDFS.subClassOf, parent_class))
    if label_fr:
        graph.add((cls, RDFS.label, Literal(label_fr, lang="fr")))
    if comment_fr:
        graph.add((cls, RDFS.comment, Literal(comment_fr, lang="fr")))
    return cls

# Fonction pour créer une propriété de données
def create_data_property(name, domain, range_type=XSD.string):
    prop = pain[name]
    graph.add((prop, RDF.type, OWL.DatatypeProperty))
    graph.add((prop, RDFS.domain, pain[domain]))
    graph.add((prop, RDFS.range, range_type))
    return prop

# === Création des classes ===
create_class("Pain", label_fr="Pain")
create_class("Ingrédient")
create_class("OrigineAnimale", "Ingrédient")
create_class("ProduitVégétal", "Ingrédient")
create_class("Farine", "ProduitVégétal")
create_class("FruitEtLegume", "ProduitVégétal")
create_class("Autre", "Ingrédient")
create_class("Levure","Ingrédient")
create_class("Fecule", "ProduitVégétal")

create_class("MéthodeDeCuisson")
create_class("Continent")
create_class("Amerique", "Continent")
create_class("Europe", "Continent")
create_class("Asie", "Continent")
create_class("Accompagnement")
create_class("Salé", "Accompagnement")
create_class("Sucré", "Accompagnement")
create_class("Seul", "Accompagnement")
create_class("Salé_et_Sucré", "Accompagnement")
create_class("RestrictionAlimentaire")
create_class("Vegan", "RestrictionAlimentaire")
create_class("SansLactose", "RestrictionAlimentaire")
create_class("SansGluten", "RestrictionAlimentaire")
create_class("Omnivore", "RestrictionAlimentaire")

# === Propriétés pour la classe Pain ===
create_data_property("alias", "Pain")
create_data_property("image", "Pain", XSD.anyURI)
create_data_property("recette", "Pain", XSD.anyURI)
create_data_property("tempsTotal", "Pain", XSD.integer)
create_data_property("tempsCuisson", "Pain", XSD.integer)
create_data_property("tempsPréparation", "Pain", XSD.integer)
create_data_property("tempsCuissonQualitatif", "Pain")
create_data_property("tempsPréparationQualitatif", "Pain")
create_data_property("tempsTotalQualitatif", "Pain")


# Relations (propriétés objet)
create_object_property("aIngrédient", "Pain", "Ingrédient")
create_object_property("aMéthodeDeCuisson", "Pain", "MéthodeDeCuisson")
create_object_property("seConsommeAvec", "Pain", "Accompagnement")

#fonction pour inférer le tps court,long ou moyen
def infer_durations(pain_uri):
    cuisson = None
    preparation = None

    for _, _, value in graph.triples((pain_uri, pain.tempsCuisson, None)):
        cuisson = int(value)

    for _, _, value in graph.triples((pain_uri, pain.tempsPréparation, None)):
        preparation = int(value)

    if cuisson is not None:
        cuisson_qual = "longue" if cuisson > 60 else "courte"
        graph.add((pain_uri, pain.tempsCuissonQualitatif, Literal(cuisson_qual, lang="fr")))

    if preparation is not None:
        prep_qual = "longue" if preparation > 30 else "courte"
        graph.add((pain_uri, pain.tempsPréparationQualitatif, Literal(prep_qual, lang="fr")))

    if cuisson is not None and preparation is not None:
        total = cuisson + preparation
        graph.set((pain_uri, pain.tempsTotal, Literal(total, datatype=XSD.integer)))
        if total < 60:
            total_qual = "court"
        elif total <= 90:
            total_qual = "moyen"
        else:
            total_qual = "long"
        graph.add((pain_uri, pain.tempsTotalQualitatif, Literal(total_qual, lang="fr")))

#Fonction pour vérifier classe et sous-classe
def has_type_or_subtype(entity, target_type):
    # Vérifie si entity a target_type ou un de ses sous-types
    for t in graph.objects(entity, RDF.type):
        if t == target_type:
            return True
        for _, _, super_class in graph.triples((t, RDFS.subClassOf, None)):
            if super_class == target_type:
                return True
    return False


#fonction pr inférer les restriction alimentaire
def infer_restriction_alimentaire(pain_uri):
    contient_animaux = False
    contient_lactose = False
    contient_gluten = False

    for _, _, ingr in graph.triples((pain_uri, pain.aIngrédient, None)):
        # Vérifie origine animale
        if has_type_or_subtype(ingr, pain.OrigineAnimale):
            contient_animaux = True
        # Vérifie lactose
        if ingr == pain["beurre"] or ingr == pain["lait"] or ingr == pain["fromage"]:
            contient_lactose = True
        # Vérifie gluten
        if ingr== pain["farine_de_ble"] or ingr== pain["farine_complète"]:
            contient_gluten = True

    # Inference des restrictions alimentaires pour le pain lui-même
    if not contient_animaux and not contient_lactose and not contient_gluten:
        graph.add((pain_uri, RDF.type, pain.Vegan))
        graph.add((pain_uri, RDF.type, pain.SansLactose))
        graph.add((pain_uri, RDF.type, pain.SansGluten))
    elif not contient_animaux and not contient_lactose:
        graph.add((pain_uri, RDF.type, pain.Vegan))
        graph.add((pain_uri, RDF.type, pain.SansLactose))
    elif not contient_animaux:
        graph.add((pain_uri, RDF.type, pain.Vegan))
    else:
        graph.add((pain_uri, RDF.type, pain.Omnivore))
        if not contient_lactose:
            graph.add((pain_uri, RDF.type, pain.SansLactose))
        if not contient_gluten:
            graph.add((pain_uri, RDF.type, pain.SansGluten))



# Fonction d'ajout d'ingrédient si non déjà défini
def add_ingredient_if_not_exists(name, type_):
    uri = pain[name]
    existing_types = list(graph.objects(uri, RDF.type))
    if not existing_types:
        graph.add((uri, RDF.type, type_))
    elif type_ not in existing_types:
        print(f"⚠️ Avertissement : {name} a déjà des types différents définis : {existing_types}")
    return uri



# realisation des instances

######################################## AMERIQUE##########################################
anadama= pain["anadama"]
graph.add((anadama, RDF.type, pain.Pain))
graph.add((anadama, RDFS.label, Literal("Anadama", lang="fr")))
#graph.add((anadama, pain.alias, Literal("Pain français classique", lang="fr"))) ce pain n'a pas d'alias
graph.add((anadama, pain.image, Literal("https://drive.google.com/file/d/1xwmw-GISJRsjTMjGwG3l6qvfGQgg4RDP/view?usp=drive_link", datatype=XSD.anyURI)))
graph.add((anadama, pain.recette, Literal("https://www.simplyrecipes.com/recipes/anadama_bread/", datatype=XSD.anyURI)))

# Temps (en minutes)
graph.add((anadama, pain.tempsCuisson, Literal(50, datatype=XSD.integer)))
graph.add((anadama, pain.tempsPréparation, Literal(40, datatype=XSD.integer)))
graph.add((anadama, pain.tempsTotal, Literal(90, datatype=XSD.integer)))


# Ajout d'ingrédients
farine = pain["farine_de_ble"]
graph.add((farine, RDF.type, pain.Farine))
graph.add((anadama, pain.aIngrédient, farine))

four= pain["four"]
graph.add((four, RDF.type, pain.MéthodeDeCuisson))
graph.add((anadama, pain.aMéthodeDeCuisson, four))

eau = pain["eau"]
graph.add((eau, RDF.type, pain.Autre))
graph.add((anadama, pain.aIngrédient, eau))

semoule_de_mais = pain["semoule_de_mais"]
graph.add((semoule_de_mais, RDF.type, pain.Fecule))
graph.add((anadama, pain.aIngrédient, semoule_de_mais))

levure = pain["levure"]
graph.add((levure, RDF.type, pain.Levure))
graph.add((anadama, pain.aIngrédient, levure))

sel = pain["sel"]
graph.add((sel, RDF.type, pain.Autre))
graph.add((anadama, pain.aIngrédient, sel))

melasse_noire= pain["mélasse_noire"]
graph.add((melasse_noire, RDF.type, pain.Autre))
graph.add((anadama, pain.aIngrédient, melasse_noire))

beurre= pain["beurre"]
graph.add((beurre, RDF.type, pain.OrigineAnimale))
graph.add((anadama, pain.aIngrédient, beurre))
#origine
etats_unis= pain["etats_unis"]
graph.add((etats_unis, RDF.type, pain.Amerique))
graph.add((anadama, pain.vientDe, etats_unis))

#accompagnement
sale_et_sucre= pain["sale_et_sucre"]
graph.add((sale_et_sucre, RDF.type, pain.Salé_et_Sucré))
graph.add((anadama, pain.seConsommeAvec, sale_et_sucre))

#inference
infer_durations(anadama)
infer_restriction_alimentaire(anadama)

carrot= pain["Carrot_bread"]
graph.add((carrot, RDF.type, pain.Pain))
graph.add((carrot, RDFS.label, Literal("Carrot_bread", lang="fr")))
#graph.add((anadama, pain.alias, Literal("Pain français classique", lang="fr"))) ce pain n'a pas d'alias
graph.add((carrot, pain.image, Literal("https://drive.google.com/drive/folders/17UybPC1rTA9Dqs4WEt7o102L4QNJuzTP", datatype=XSD.anyURI)))
graph.add((carrot, pain.recette, Literal("https://poppycuisine.blogspot.com/2012/04/carrot-bread-la-version-saine-du-carrot.html", datatype=XSD.anyURI)))

# Temps (en minutes)
graph.add((carrot, pain.tempsCuisson, Literal(60, datatype=XSD.integer)))
graph.add((carrot, pain.tempsPréparation, Literal(15, datatype=XSD.integer)))
graph.add((carrot, pain.tempsTotal, Literal(75, datatype=XSD.integer)))


# Ajout d'ingrédients
farine = pain["farine_de_ble"]
graph.add((carrot, pain.aIngrédient, farine))

four= pain["four"]
graph.add((carrot, pain.aMéthodeDeCuisson, four))

carot= pain["carotte"]
graph.add((carot, RDF.type, pain.FruitEtLegume))
graph.add((carrot, pain.aIngrédient, carot))

farine_comp = pain["farine_complète"]
graph.add((farine_comp, RDF.type, pain.Farine))
graph.add((carrot, pain.aIngrédient, farine_comp))

levure = pain["levure"]
graph.add((carrot, pain.aIngrédient, levure))

avoine= pain["flocons_d_avoine"]
graph.add((avoine, RDF.type, pain.Farine))
graph.add((carrot, pain.aIngrédient, avoine))

sucre_roux= pain["sucre_roux"]
graph.add((sucre_roux, RDF.type, pain.Autre))
graph.add((carrot, pain.aIngrédient, sucre_roux))

lait= pain["lait"]
graph.add((lait, RDF.type, pain.OrigineAnimale))
graph.add((carrot, pain.aIngrédient, lait))

orange= pain["jus_orange"]
graph.add((orange, RDF.type, pain.FruitEtLegume))
graph.add((carrot, pain.aIngrédient, orange))

amande= pain["amande_effilées"]
graph.add((amande, RDF.type, pain.Autre))
graph.add((carrot, pain.aIngrédient, amande))

noix= pain["noix"]
graph.add((noix, RDF.type, pain.Autre))
graph.add((carrot, pain.aIngrédient, noix))

zesteorange= pain["zeste_orange"]
graph.add((zesteorange, RDF.type, pain.FruitEtLegume))
graph.add((carrot, pain.aIngrédient, zesteorange))

zestecitron= pain["zeste_citron"]
graph.add((zestecitron, RDF.type, pain.FruitEtLegume))
graph.add((carrot, pain.aIngrédient, zestecitron))

oeuf= pain["oeuf"]
graph.add((oeuf, RDF.type, pain.OrigineAnimale))
graph.add((oeuf, pain.aIngrédient, oeuf))

#origine
etats_unis= pain["etats_unis"]
graph.add((etats_unis, RDF.type, pain.Amerique))
graph.add((carrot, pain.vientDe, etats_unis))

#accompagnement
seul= pain["seul"]
graph.add((seul, RDF.type, pain.Seul))
graph.add((carrot, pain.seConsommeAvec, seul))

#inference
infer_durations(carrot)
infer_restriction_alimentaire(carrot)

#=== montreal bagel===

montreal_bagel = pain["montreal_bagel"]
graph.add((montreal_bagel, RDF.type, pain.Pain))

# Ingrédients
farine = add_ingredient_if_not_exists("farine_de_ble", pain.Farine)
eau = add_ingredient_if_not_exists("eau", pain.Eau)
levure = add_ingredient_if_not_exists("levure", pain.Levain)
sucre = add_ingredient_if_not_exists("sucre", pain.Autre)
sel = add_ingredient_if_not_exists("sel", pain.Sel)
oeuf = add_ingredient_if_not_exists("oeuf", pain.OrigineAnimale)
miel = add_ingredient_if_not_exists("miel", pain.OrigineAnimale)

for ingr in [farine, eau, levure, sucre, sel, oeuf, miel]:
    graph.add((montreal_bagel, pain.aIngrédient, ingr))
canada=pain["canada"]
graph.add((canada, RDF.type, pain.Amerique))
graph.add((montreal_bagel, pain.vientDe, canada))
graph.add((montreal_bagel, pain.seConsommeAvec, sale_et_sucre))
bouilli= pain["bouilli"]
graph.add((bouilli, RDF.type, pain.MéthodeDeCuisson))
graph.add((montreal_bagel, pain.aMéthodeDeCuisson, four))
graph.add((montreal_bagel, pain.aMéthodeDeCuisson, bouilli))
graph.add((montreal_bagel, pain.image, Literal("https://drive.google.com/drive/folders/17UybPC1rTA9Dqs4WEt7o102L4QNJuzTP", datatype=XSD.anyURI)))
graph.add((montreal_bagel, pain.recette, Literal("https://catherinecuisine.org/2018/01/19/bagels-de-montreal/", datatype=XSD.anyURI)))

infer_durations(montreal_bagel)
infer_restriction_alimentaire(montreal_bagel)

# === Chipa ===
chipa = pain["chipa"]
graph.add((chipa, RDF.type, pain.Pain))

fecule_manioc = add_ingredient_if_not_exists("fecule_de_manioc", pain.Fecule)
fromage = add_ingredient_if_not_exists("fromage", pain.OrigineAnimale)
lait = add_ingredient_if_not_exists("lait", pain.OrigineAnimale)
beurre = add_ingredient_if_not_exists("beurre", pain.OrigineAnimale)

for ingr in [fecule_manioc, oeuf, fromage, lait, beurre]:
    graph.add((chipa, pain.aIngrédient, ingr))

paraguay= pain["paraguay"]
graph.add((paraguay, RDF.type, pain.Amerique))
graph.add((chipa, pain.vientDe, paraguay))
graph.add((chipa, pain.seConsommeAvec, seul))
graph.add((chipa, pain.aMéthodeDeCuisson, four))
infer_durations(chipa)
infer_restriction_alimentaire(chipa)
graph.add((chipa, pain.image, Literal("https://drive.google.com/drive/folders/17UybPC1rTA9Dqs4WEt7o102L4QNJuzTP", datatype=XSD.anyURI)))
graph.add((chipa, pain.recette, Literal("https://www.cuisineaz.com/recettes/chipa-105794.aspx", datatype=XSD.anyURI)))

# === Sopa paraguaya ===
sopa = pain["sopa_paraguaya"]
graph.add((sopa, RDF.type, pain.Pain))

semoule_mais = add_ingredient_if_not_exists("semoule_de_mais", pain.Farine)
oignon = add_ingredient_if_not_exists("oignon", pain.FruitEtLegume)

for ingr in [semoule_mais, oeuf, fromage, lait, oignon]:
    graph.add((sopa, pain.aIngrédient, ingr))
graph.add((sopa, pain.vientDe, paraguay))
graph.add((sopa, pain.seConsommeAvec, seul))
graph.add((chipa, pain.aMéthodeDeCuisson, four))
graph.add((sopa, pain.image, Literal("https://drive.google.com/drive/folders/17UybPC1rTA9Dqs4WEt7o102L4QNJuzTP", datatype=XSD.anyURI)))
graph.add((sopa, pain.recette, Literal("https://www.cuisineaz.com/recettes/sopa-paraguaya-9548.aspx", datatype=XSD.anyURI)))
infer_durations(sopa)
infer_restriction_alimentaire(sopa)

 #######################################################################################
graph.serialize(destination="ontologie_pain.owl", format="turtle")




⚠️ Avertissement : eau a déjà des types différents définis : [rdflib.term.URIRef('http://example.org/ontopain#Autre')]
⚠️ Avertissement : levure a déjà des types différents définis : [rdflib.term.URIRef('http://example.org/ontopain#Levure')]
⚠️ Avertissement : sel a déjà des types différents définis : [rdflib.term.URIRef('http://example.org/ontopain#Autre')]
⚠️ Avertissement : semoule_de_mais a déjà des types différents définis : [rdflib.term.URIRef('http://example.org/ontopain#Fecule')]


<Graph identifier=N8ef7ed1e6c314772bd92139572b9054c (<class 'rdflib.graph.Graph'>)>